<h1 style="text-align:center;">Setting the ESKM username and password for HPE iLO systems</h1></div>

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This Jupyter Notebook will contains a example code to demonstrate of Setting the ESKM username and password for HPE iLO systems. This code is written in Python and uses the redfish library to perform the required operations.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this u need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
    </ul>
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory
#This is to take passwords as input securely
import getpass

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 2. This is about ESKM_username_pass function</h3>
        
1. The function sets up an Electronic Software Key Manager (ESKM) on an iLO (Integrated Lights Out) server using the Redfish API.<br>
<br>
    
2. It takes several input parameters including ESKM username, password, account group, primary key server address, and primary key server port.<br>
    <br>

3. It first attempts to locate the URI for the ESKM resource either by using the Redfish Resource Directory or by manually navigating through the Redfish API.br>
    <br>

4. If the ESKM URI is found, the function constructs a JSON payload containing the ESKM configuration information and primary key server details and sends it as a PATCH request to the ESKM resource.<br>
    <br>
5.  If the PATCH request is successful (i.e., returns an HTTP status code of 200), the function prints out a success message along with the response payload, otherwise it prints out an error message.
<br>
    



 </div>

In [1]:
def ESKM_username_pass(_redfishobj, eskm_username, eskm_password, eskm_accountgroup,eskm_primarykeyserver_addr, eskm_primarykeyserver_port):

    eskm_uri = None
    body = dict()

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        security_service_uri = managers_members_response.obj.Oem.Hpe.Links['SecurityService']['@odata.id']
                                                                                    
        security_service_response = _redfishobj.get(security_service_uri)
        eskm_uri = security_service_response.obj.Links['ESKM']['@odata.id']
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#HpeESKM.' in instance['@odata.type']:
                eskm_uri = instance['@odata.id']
                break

    if eskm_uri:
        body["KeyManagerConfig"] = dict()
        body["KeyManagerConfig"]["LoginName"] = eskm_username
        body["KeyManagerConfig"]["Password"] = eskm_password
        body["KeyManagerConfig"]["AccountGroup"] = eskm_accountgroup
        body["KeyManagerConfig"]["ESKMLocalCACertificateName"] = ""
        body["PrimaryKeyServerAddress"] = eskm_primarykeyserver_addr
        body["PrimaryKeyServerPort"] = eskm_primarykeyserver_port
        resp = _redfishobj.patch(eskm_uri, body)
        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
                                                                               
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended Message Info...")
                                 
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 3. This is Main function overview.</h3>
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE TO PROCEED FURTHER<br>
    <ul>
        <li>Once your credentials that are entered a redfish obejct is created with the help of RedfishClient.</li>
        <li>If server is down or Unreachable we get server not reachable message!</li>
        <li>If not then we make a call to <b>mount_virtual_media_iso</b> to obtain virtual media related info of system.</li>
        <li>After getting required information <i><b>Redfish.logout()</b></i> closses the particular session and logs us out of the system.</li>
    </ul>
</div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally,</b>use the following values:<br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address, account name and password to send https requests<br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

In [2]:
if __name__ == "__main__":
    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Login account: ")
    LOGIN_PASSWORD = getpass.getpass("Enter your password: ")

    ESKM_USERNAME = input("Enter username")
    ESKM_PASSWORD = input("Enter password")
    ESKM_ACCOUNTGROUP = input("Enter account group")
    ESKM_PRIMARYKEYSERVER_ADDR = input("Enter server address")
    ESKM_PRIMARYKEYSERVER_PORT = int(input("Enter server port"))
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT,password=LOGIN_PASSWORD)
                                                                            
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    ESKM_username_pass(REDFISHOBJ, ESKM_USERNAME, ESKM_PASSWORD, ESKM_ACCOUNTGROUP,ESKM_PRIMARYKEYSERVER_ADDR, ESKM_PRIMARYKEYSERVER_PORT)
    REDFISHOBJ.logout()

NameError: name 'ServerDownOrUnreachableError' is not defined

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.REST API: <a href = "https://restfulapi.net/">LINK</a><br>
    4.Changing Boot Setting Documentation: <a href = "https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/?python#changing-boot-settings">LINK</a><br>
    
</div>